In [1]:
!pip install flask flask_sqlalchemy flask_ngrok


In [2]:
import os
if os.path.exists("ems.db"):
    os.remove("ems.db")
    print("Old database deleted.")
else:
    print("No existing database found.")


No existing database found.


In [3]:
from flask import Flask, request, redirect, render_template_string
import sqlite3

app = Flask(__name__)



## --- Initialize Database ---


In [4]:
conn = sqlite3.connect("ems.db")
cur = conn.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS roles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    role_name TEXT NOT NULL
)''')

cur.execute('''CREATE TABLE IF NOT EXISTS employees (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT NOT NULL,
    role_id INTEGER,
    FOREIGN KEY (role_id) REFERENCES roles(id)
)''')

cur.execute('''CREATE TABLE IF NOT EXISTS attendance (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    employee_id INTEGER,
    date TEXT,
    status TEXT,
    FOREIGN KEY (employee_id) REFERENCES employees(id)
)''')

cur.execute('''CREATE TABLE IF NOT EXISTS payroll (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    employee_id INTEGER,
    salary REAL,
    bonus REAL,
    FOREIGN KEY (employee_id) REFERENCES employees(id)
)''')

conn.commit()
conn.close()


# --- HTML Layout ---

In [5]:
layout = '''
<h2>Employment Management System</h2>
<nav>
    <a href="/">Home</a> |
    <a href="/employees">Employees</a> |
    <a href="/attendance">Attendance</a> |
    <a href="/payroll">Payroll</a> |
    <a href="/roles">Roles</a>
</nav><hr>
'''

# --- Home with Predefined Queries ---
@app.route('/')
def home():
    return render_template_string(layout + '''
        <h3>Welcome to EMS</h3>
        <p><b>Predefined Queries:</b></p>
        <ul>
            <li><a href="/high_salary">Employees with salary > 50000</a></li>
            <li><a href="/present_today">Employees present today</a></li>
            <li><a href="/employee_count_by_role">Employee count by role</a></li>
            <li><a href="/bonus_more_than_10k">Employees with bonus > 10000</a></li>
        </ul>
    ''')

@app.route('/high_salary')
def high_salary():
    conn = sqlite3.connect("ems.db")
    cur = conn.cursor()
    data = cur.execute('''SELECT employees.name, salary FROM payroll
                          JOIN employees ON payroll.employee_id = employees.id
                          WHERE salary > 50000''').fetchall()
    conn.close()
    return render_template_string(layout + '''
        <h3>Employees with Salary > 50000</h3>
        <ul>{% for row in data %}
            <li>{{ row[0] }} - ₹{{ row[1] }}</li>
        {% endfor %}</ul>
    ''', data=data)

@app.route('/present_today')
def present_today():
    import datetime
    today = datetime.date.today().isoformat()
    conn = sqlite3.connect("ems.db")
    cur = conn.cursor()
    data = cur.execute('''SELECT employees.name, date FROM attendance
                          JOIN employees ON attendance.employee_id = employees.id
                          WHERE date = ? AND status = 'Present' ''', (today,)).fetchall()
    conn.close()
    return render_template_string(layout + '''
        <h3>Employees Present Today ({{ today }})</h3>
        <ul>{% for row in data %}
            <li>{{ row[0] }} - {{ row[1] }}</li>
        {% endfor %}</ul>
    ''', data=data, today=today)

@app.route('/employee_count_by_role')
def employee_count_by_role():
    conn = sqlite3.connect("ems.db")
    cur = conn.cursor()
    data = cur.execute('''SELECT role_name, COUNT(employees.id) FROM roles
                          LEFT JOIN employees ON roles.id = employees.role_id
                          GROUP BY role_name''').fetchall()
    conn.close()
    return render_template_string(layout + '''
        <h3>Employee Count by Role</h3>
        <ul>{% for row in data %}
            <li>{{ row[0] }} - {{ row[1] }} employees</li>
        {% endfor %}</ul>
    ''', data=data)

@app.route('/bonus_more_than_10k')
def bonus_more_than_10k():
    conn = sqlite3.connect("ems.db")
    cur = conn.cursor()
    data = cur.execute('''SELECT employees.name, bonus FROM payroll
                          JOIN employees ON payroll.employee_id = employees.id
                          WHERE bonus > 10000''').fetchall()
    conn.close()
    return render_template_string(layout + '''
        <h3>Employees with Bonus > ₹10000</h3>
        <ul>{% for row in data %}
            <li>{{ row[0] }} - ₹{{ row[1] }}</li>
        {% endfor %}</ul>
    ''', data=data)


# --- Employees ---

In [6]:
@app.route('/employees', methods=['GET', 'POST'])
def employees():
    conn = sqlite3.connect("ems.db")
    cur = conn.cursor()
    if request.method == 'POST':
        name = request.form['name']
        email = request.form['email']
        role_id = request.form['role_id']
        cur.execute("INSERT INTO employees (name, email, role_id) VALUES (?, ?, ?)", (name, email, role_id))
        conn.commit()
    employees = cur.execute('''SELECT employees.id, name, email, role_name
                               FROM employees LEFT JOIN roles ON employees.role_id = roles.id''').fetchall()
    roles = cur.execute("SELECT id, role_name FROM roles").fetchall()
    conn.close()
    return render_template_string(layout + '''
        <h3>Employees</h3>
        <form method="post">
            Name: <input name="name">
            Email: <input name="email">
            Role: <select name="role_id">
                {% for r in roles %}<option value="{{r[0]}}">{{r[1]}}</option>{% endfor %}
            </select>
            <input type="submit" value="Add">
        </form><br>
        <table border=1>
            <tr><th>ID</th><th>Name</th><th>Email</th><th>Role</th><th>Action</th></tr>
            {% for e in employees %}
                <tr>
                    <td>{{e[0]}}</td><td>{{e[1]}}</td><td>{{e[2]}}</td><td>{{e[3]}}</td>
                    <td><a href="/delete_employee/{{e[0]}}">Delete</a></td>
                </tr>
            {% endfor %}
        </table>
    ''', employees=employees, roles=roles)

@app.route('/delete_employee/<int:id>')
def delete_employee(id):
    conn = sqlite3.connect("ems.db")
    cur = conn.cursor()
    cur.execute("DELETE FROM employees WHERE id=?", (id,))
    conn.commit()
    conn.close()
    return redirect('/employees')

# --- Attendance ---

In [7]:
@app.route('/attendance', methods=['GET', 'POST'])
def attendance():
    conn = sqlite3.connect("ems.db")
    cur = conn.cursor()
    if request.method == 'POST':
        emp_id = request.form['employee_id']
        date = request.form['date']
        status = request.form['status']
        cur.execute("INSERT INTO attendance (employee_id, date, status) VALUES (?, ?, ?)", (emp_id, date, status))
        conn.commit()
    records = cur.execute('''SELECT attendance.id, employees.name, date, status
                             FROM attendance JOIN employees ON attendance.employee_id = employees.id''').fetchall()
    employees = cur.execute("SELECT id, name FROM employees").fetchall()
    conn.close()
    return render_template_string(layout + '''
        <h3>Attendance</h3>
        <form method="post">
            Employee: <select name="employee_id">
                {% for emp in employees %}<option value="{{emp[0]}}">{{emp[1]}}</option>{% endfor %}
            </select>
            Date: <input name="date" type="date">
            Status: <input name="status">
            <input type="submit" value="Add">
        </form><br>
        <table border=1>
            <tr><th>ID</th><th>Employee</th><th>Date</th><th>Status</th><th>Action</th></tr>
            {% for a in records %}
                <tr>
                    <td>{{a[0]}}</td><td>{{a[1]}}</td><td>{{a[2]}}</td><td>{{a[3]}}</td>
                    <td><a href="/delete_attendance/{{a[0]}}">Delete</a></td>
                </tr>
            {% endfor %}
        </table>
    ''', records=records, employees=employees)

@app.route('/delete_attendance/<int:id>')
def delete_attendance(id):
    conn = sqlite3.connect("ems.db")
    cur = conn.cursor()
    cur.execute("DELETE FROM attendance WHERE id=?", (id,))
    conn.commit()
    conn.close()
    return redirect('/attendance')

# --- Payroll ---

In [8]:
@app.route('/payroll', methods=['GET', 'POST'])
def payroll():
    conn = sqlite3.connect("ems.db")
    cur = conn.cursor()
    if request.method == 'POST':
        emp_id = request.form['employee_id']
        salary = request.form['salary']
        bonus = request.form['bonus']
        cur.execute("INSERT INTO payroll (employee_id, salary, bonus) VALUES (?, ?, ?)", (emp_id, salary, bonus))
        conn.commit()
    data = cur.execute('''SELECT payroll.id, employees.name, salary, bonus
                          FROM payroll JOIN employees ON payroll.employee_id = employees.id''').fetchall()
    employees = cur.execute("SELECT id, name FROM employees").fetchall()
    conn.close()
    return render_template_string(layout + '''
        <h3>Payroll</h3>
        <form method="post">
            Employee: <select name="employee_id">
                {% for emp in employees %}<option value="{{emp[0]}}">{{emp[1]}}</option>{% endfor %}
            </select>
            Salary: <input name="salary" type="number">
            Bonus: <input name="bonus" type="number">
            <input type="submit" value="Add">
        </form><br>
        <table border=1>
            <tr><th>ID</th><th>Employee</th><th>Salary</th><th>Bonus</th><th>Action</th></tr>
            {% for row in data %}
                <tr>
                    <td>{{row[0]}}</td><td>{{row[1]}}</td><td>{{row[2]}}</td><td>{{row[3]}}</td>
                    <td><a href="/delete_payroll/{{row[0]}}">Delete</a></td>
                </tr>
            {% endfor %}
        </table>
    ''', data=data, employees=employees)

@app.route('/delete_payroll/<int:id>')
def delete_payroll(id):
    conn = sqlite3.connect("ems.db")
    cur = conn.cursor()
    cur.execute("DELETE FROM payroll WHERE id=?", (id,))
    conn.commit()
    conn.close()
    return redirect('/payroll')

# --- Roles ---

In [9]:
@app.route('/roles', methods=['GET', 'POST'])
def roles():
    conn = sqlite3.connect("ems.db")
    cur = conn.cursor()
    if request.method == 'POST':
        role_name = request.form['role_name']
        cur.execute("INSERT INTO roles (role_name) VALUES (?)", (role_name,))
        conn.commit()
    roles = cur.execute("SELECT * FROM roles").fetchall()
    conn.close()
    return render_template_string(layout + '''
        <h3>Roles</h3>
        <form method="post">
            Role: <input name="role_name">
            <input type="submit" value="Add">
        </form><br>
        <table border=1>
            <tr><th>ID</th><th>Role Name</th><th>Action</th></tr>
            {% for r in roles %}
                <tr>
                    <td>{{r[0]}}</td><td>{{r[1]}}</td>
                    <td><a href="/delete_role/{{r[0]}}">Delete</a></td>
                </tr>
            {% endfor %}
        </table>
    ''', roles=roles)

@app.route('/delete_role/<int:id>')
def delete_role(id):
    conn = sqlite3.connect("ems.db")
    cur = conn.cursor()
    cur.execute("DELETE FROM roles WHERE id=?", (id,))
    conn.commit()
    conn.close()
    return redirect('/roles')

In [ ]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Apr/2025 09:09:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 09:09:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Apr/2025 09:10:12] "GET /employees HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 09:10:33] "GET /roles HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 09:10:38] "POST /roles HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 09:10:42] "POST /roles HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 09:10:48] "POST /roles HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 09:11:05] "GET /employees HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 09:11:14] "POST /employees HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 09:11:28] "POST /employees HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 09:11:38] "POST /employees HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 09:11:54] "POST /employees HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 09:12:13] "POST /employees HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2025 09:12:31] "POST /employees HTTP/1.